In [2]:
import numpy as np
import pandas as pd
import pandas_profiling

In [3]:
titanicRawTrain = pd.read_csv('../data/raw/train.csv')

## Load Rob's Titanic cleaning and feature engineering

In [4]:
import sys
sys.path.append('/home/rob/Dropbox/DataProject/titanicKaggle/src/features')

In [5]:
import build_features

In [6]:
titanicCleanTrain = build_features.execute_cleaning(titanicRawTrain)

In [7]:
titanicColumns = build_features.produce_columns(titanicCleanTrain)

In [8]:
titanicCleanTrain.head()

,PassengerId,Survived,Parch,Age_Unknown,Age_Baby,Age_Child,Age_Teenager,Age_Student,Age_Young Adult,Age_Adult,...,Fare_4_quartile,Class_1,Class_2,Class_3,Sex_female,Sex_male,Sibs_Small,Sibs_Medium,Sibs_Large,Sibs_ExtLarge
0,1,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,1,1,0,0,0
1,2,1,0,0,0,0,0,0,0,1,...,1,1,0,0,1,0,1,0,0,0
2,3,1,0,0,0,0,0,0,1,0,...,0,0,0,1,1,0,0,0,0,0
3,4,1,0,0,0,0,0,0,1,0,...,1,1,0,0,1,0,1,0,0,0
4,5,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,1,0,0,0,0


In [9]:
titanicCleanTrain.columns

Index(['PassengerId', 'Survived', 'Parch', 'Age_Unknown', 'Age_Baby',
       'Age_Child', 'Age_Teenager', 'Age_Student', 'Age_Young Adult',
       'Age_Adult', 'Age_Senior', 'Embarked_C', 'Embarked_Q', 'Embarked_S',
       'Fare_Unknown', 'Fare_1_quartile', 'Fare_2_quartile', 'Fare_3_quartile',
       'Fare_4_quartile', 'Class_1', 'Class_2', 'Class_3', 'Sex_female',
       'Sex_male', 'Sibs_Small', 'Sibs_Medium', 'Sibs_Large', 'Sibs_ExtLarge'],
      dtype='object')

## Investigate variability of factors

## Load new model

In [10]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.model_selection import cross_val_score

In [11]:
X_train, X_test, y_train, y_test = train_test_split(titanicCleanTrain[titanicColumns], 
                                                    titanicCleanTrain.Survived, test_size = 0.4, random_state =0)

In [12]:
clf = linear_model.LogisticRegression().fit(X_train, y_train)

In [13]:
clf.score(X_test, y_test)

0.82352941176470584

In [18]:
from sklearn.cross_validation import KFold
from sklearn.metrics import make_scorer, accuracy_score

/home/rob/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [19]:
def run_kfold(clf, X_all, y_all):
    kf = KFold(891, n_folds = 10)
    outcomes = []
    fold = 0
    for train_i, test_i in kf:
        fold += 1
        X_train, X_test = X_all.values[train_i], X_all.values[test_i]
        y_train, y_test = y_all.values[train_i], y_all.values[test_i]
        clf.fit(X_train, y_train)
        predictions = clf.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        outcomes.append(accuracy)
        print("Fold {0} accuracy: {1}".format(fold, accuracy))
    mean_outcome = np.mean(outcomes)
    print("Mean Accuracy: {0}". format(mean_outcome))

In [20]:
run_kfold(clf, titanicCleanTrain[titanicColumns], titanicCleanTrain['Survived'])

Fold 1 accuracy: 0.7888888888888889
Fold 2 accuracy: 0.8202247191011236
Fold 3 accuracy: 0.8089887640449438
Fold 4 accuracy: 0.8539325842696629
Fold 5 accuracy: 0.8089887640449438
Fold 6 accuracy: 0.797752808988764
Fold 7 accuracy: 0.7752808988764045
Fold 8 accuracy: 0.7865168539325843
Fold 9 accuracy: 0.8764044943820225
Fold 10 accuracy: 0.8539325842696629
Mean Accuracy: 0.8170911360799001


## Test it out

In [21]:
titanicRawTest = pd.read_csv('../data/raw/test.csv')

In [23]:
data_test = build_features.execute_cleaning(titanicRawTest)

In [24]:
ids = data_test['PassengerId']
predictions = clf.predict(data_test.drop('PassengerId', axis=1))

output = pd.DataFrame({ 'PassengerId' : ids, 'Survived': predictions })
output.to_csv('titanic-predictions.csv', index = False)

In [25]:
output.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,0


## ToDo:
New features (ANOVA investigation)

Normalization

